In [41]:
import pandas as pd
import datetime

# Import data

In [90]:
df_client=pd.read_csv("./raw_data/client.csv")
df_electricity=pd.read_csv("./raw_data/electricity_prices.csv")
df_gas=pd.read_csv("./raw_data/gas_prices.csv")



# Changing data format

### Manipulating dates
Check the number of unique values for the date column in the electricity
By checking only the unique dates, it will reduce the number of calculation when we will compare the date between the electricity df and the train df

In [91]:
print("Number of unique values in elec :", df_electricity["origin_date"].nunique())
print("Number of unique values in gas :", df_gas["origin_date"].nunique())

Number of unique values in elec : 15286
Number of unique values in gas : 637


In [44]:
# # Creating a new column "date" with the French formated date 
# # For elec
# df_electricity["date"] = pd.to_datetime(df_electricity["forecast_date"])
# df_electricity["date"] = df_electricity["date"].apply(lambda x : x.strftime("%y-%m-%d"))
# print(df_electricity["date"])
# 
# # For gases
# df_gas["date"] = pd.to_datetime(df_gas["forecast_date"])
# df_gas["date"] = df_gas["date"].apply(lambda x : x.strftime("%y-%m-%d"))
# print(df_gas["date"])

# Feature Engineering

### Adding prices in df_train

In [96]:
# Import train data
df_train = pd.read_csv("./raw_data/train.csv").sample(10000)

## Merge elec prices
# Creating a new column based on datetime
df_train["origin_date"] = df_train["datetime"]
# Selecting the infos we need to merge in the electricity df
df_elec_prices = df_electricity.filter(['euros_per_mwh', 'origin_date'], axis=1)
# Merging
df_train = df_train.merge(df_elec_prices, how="left", on="origin_date")

## Merge gas prices
# Removing the hours in the origin_date column created above
df_train["origin_date"] = df_train["origin_date"].apply(lambda x: pd.to_datetime(x).strftime("%Y-%m-%d"))
# Selecting the infos we need to merge in the gas df
df_gas_prices = df_gas.filter(['lowest_price_per_mwh', 'highest_price_per_mwh', 'origin_date'])
# Merging
df_train = df_train.merge(df_gas_prices, how="left", on="origin_date")

# df_train.describe()
print(df_train)

      county  is_business  product_type    target  is_consumption  \
0          1            1             3    25.907               1   
1          4            0             3   112.098               1   
2         14            0             3     0.000               0   
3          4            1             0   443.366               1   
4          7            1             0     0.000               0   
...      ...          ...           ...       ...             ...   
9995       0            1             3  6452.831               1   
9996      13            1             1    22.931               1   
9997       5            0             3   722.581               0   
9998      12            1             3   717.017               1   
9999      10            1             3  2154.453               1   

                 datetime  data_block_id   row_id  prediction_unit_id  \
0     2021-11-04 02:00:00             64   190927                   8   
1     2023-02-26 22:00:00

In [70]:
df_train["elec_price"] = -1
df_train["low_gas_price"] = -1
df_train["high_gas_price"] = -1


for unique_date in df_train["datetime"].unique():
  # Récuperer le prix de l'electricite a ce jour la
  df_elec_filtered = df_electricity[df_electricity["origin_date"] == unique_date]
  if len(df_elec_filtered) != 0:
    if len(df_elec_filtered) == 1:
      elec_price = df_elec_filtered["euros_per_mwh"]
    else : 
      elec_price = df_elec_filtered.iloc[0]["euros_per_mwh"]

    # Pour toutes les lignes de df_train a ce jour la, insérer le prix
    df_train_filtered = df_train[df_train["datetime"] == unique_date]
    df_train.loc[df_train_filtered.index, "elec_price"] = elec_price.values[0]
  
  
  # Récuperer le prix du gas a ce jour la
  df_gas_filtered = df_gas[df_gas["date"] == pd.to_datetime(unique_date).strftime("%y-%m-%d")]
  # print(len(df_gas_filtered))
  if len(df_gas_filtered) != 0:
    if len(df_gas_filtered) == 1:
      low_gas_price = df_gas_filtered["lowest_price_per_mwh"]
      high_gas_price = df_gas_filtered["highest_price_per_mwh"]
    else : 
      low_gas_price = df_gas_filtered["lowest_price_per_mwh"].values[0]
      high_gas_price = df_gas_filtered["highest_price_per_mwh"].values[0]

    # Pour toutes les lignes de df_train a ce jour la, insérer les prix
    df_train_filtered = df_train[df_train["datetime"] == unique_date]
    df_train.loc[df_train_filtered.index, "low_gas_price"] = low_gas_price.values[0]
    df_train.loc[df_train_filtered.index, "high_gas_price"] = high_gas_price.values[0]


KeyError: 'datetime'

In [69]:
df_train

,county,is_business,product_type,target,is_consumption,origin_date,data_block_id,row_id,prediction_unit_id,euros_per_mwh_x,euros_per_mwh_y,elec_price,low_gas_price,high_gas_price
0,10,1,3,815.284,1,2023-04-23 01:00:00,599,1894009,42,36.97,36.97,-1,-1,-1
1,1,0,1,0.000,0,2021-12-02 01:00:00,92,275468,6,124.99,124.99,-1,-1,-1
2,0,1,1,0.000,0,2022-11-07 02:00:00,432,1358732,4,85.67,85.67,-1,-1,-1
3,15,1,3,1064.623,0,2022-06-23 13:00:00,295,920642,60,65.38,65.38,-1,-1,-1
4,2,1,3,306.643,1,2021-09-26 22:00:00,25,75905,10,98.14,98.14,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,13,1,3,25.252,0,2021-10-11 08:00:00,40,119196,52,188.44,188.44,-1,-1,-1
9996,5,0,3,234.712,1,2022-12-01 10:00:00,456,1439305,20,428.44,428.44,-1,-1,-1
9997,1,0,3,1.367,0,2022-12-29 20:00:00,484,1530632,7,95.09,95.09,-1,-1,-1
9998,7,0,3,32.699,1,2022-09-22 10:00:00,386,1209851,27,350.92,350.92,-1,-1,-1


In [ ]:
df_train.describe()